<a href="https://colab.research.google.com/github/whkwls2653/Emotion-Recognition/blob/main/kobert_KEMDy19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torc
##Kobert pretrained model
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
##numpy version for error
!pip install numpy==1.23.1

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
print(np.__version__)
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

1.23.1


In [69]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [70]:
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [51]:
import csv
import pandas as pd
import os

annotation_root= '/content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/annotation'
txt_label_zip_toteval_root='/content/gdrive/MyDrive/감정인식 대회/데이터셋/2019txt_label_zip_toteval.pkl'

ambig_annotation=0
no_file=0
if os.path.isfile(txt_label_zip_toteval_root):
  with open(txt_label_zip_toteval_root,'rb') as f:
    allfile_datalist=pickle.load(f)
else:
  allfile_datalist=[]
  for i, annotation in tqdm(enumerate(os.listdir(annotation_root))):
    all_txts=[]
    # print('annotaion file : (%d / %d)'%(i,len(os.listdir(annotation_root))))
    session_file = pd.read_csv(os.path.join(annotation_root,annotation)) 
    # print(session_file) 
    segments=session_file['Segment ID']
    # print(segments)
    
    for j in range(1,len(segments)):

      #index starts from 1
      f_name=segments[j]
      
      # print("f_name:",f_name)
      sess_num=f_name.split('_')[0][-2:]
      sc_pro_num=f_name.split('_')[0]+'_'+f_name.split('_')[1]
      # print(sess_num,sc_pro_num)
      
    #  /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session01/Sess01_script01/Sess01_script01_F001.txt
      file_loc='/content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session%s/%s/%s.txt'%(sess_num,sc_pro_num,f_name)
      # print(file_loc)
      if not os.path.isfile(file_loc):
        print("no file location :",file_loc)
        no_file+=1
        continue
      with open(file_loc) as f:
        lines = f.readlines()
        all_txts.append(lines)



    session_file.loc[(session_file['Total Evaluation'] == "fear"), 'Total Evaluation'] = 0  #공포 => 0
    session_file.loc[(session_file['Total Evaluation'] == "surprise"), 'Total Evaluation'] = 1  #놀람 => 1
    session_file.loc[(session_file['Total Evaluation'] == "angry"), 'Total Evaluation'] = 2  #분노 => 2
    session_file.loc[(session_file['Total Evaluation'] == "sad"), 'Total Evaluation'] = 3  #슬픔 => 3
    session_file.loc[(session_file['Total Evaluation'] == "neutral"), 'Total Evaluation'] = 4  #중립 => 4
    session_file.loc[(session_file['Total Evaluation'] == "happy"), 'Total Evaluation'] = 5  #행복 => 5
    session_file.loc[(session_file['Total Evaluation'] == "disgust"), 'Total Evaluation'] = 6  #혐오 => 6



    
    # print(len(all_txts))
    ## zip txts and labels,  session_file start from 1
    for q, label in zip(all_txts, session_file['Total Evaluation'][1:])  :
      if type(label) is not int :
        ambig_annotation+=1
        continue
      data = []
      data.append(q[0])
      # print('Q',type(q),q[0])
      # print('label',label)
      data.append(str(label))
      allfile_datalist.append(data)
    # print(data_list)
    # allfile_datalist.append(data_list)



  with open(txt_label_zip_toteval_root,'wb') as f:
    pickle.dump(allfile_datalist,f)
# print("data_list length :",len(data_list))
# print("sample : ",data_list[0])
  
# print(len(all_txts))





30it [00:18,  1.71it/s]

no file location : /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session03/Sess03_script04/Sess03_script04_M016.txt


31it [00:18,  1.65it/s]

no file location : /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session03/Sess03_script04/Sess03_script04_M016.txt


33it [00:20,  1.65it/s]

...

no file location : /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session07/Sess07_script02/Sess07_script02_M026.txt
no file location : /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session07/Sess07_script02/Sess07_script02_M027.txt


36it [00:21,  1.66it/s]

no file location : /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session07/Sess07_script02/Sess07_script02_M026.txt
no file location : /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session07/Sess07_script02/Sess07_script02_M027.txt


40it [00:24,  1.63it/s]

no file location : /content/gdrive/MyDrive/감정인식 대회/데이터셋/KEMDy19/wav/Session04/Sess04_impro03/Sess04_impro03_F031.txt


In [ ]:
print(allfile_datalist)
print(len(allfile_datalist))

In [ ]:
#### test1 \n 등빼기
if os.path.isfile(txt_label_zip_toteval_root):
  with open(txt_label_zip_toteval_root,'rb') as f:
    allfile_datalist_processed=pickle.load(f)
for words in allfile_datalist_processed: 
  
  
  if '\n' in words[0] or'/'in words[0] :
    new_words=words[0].replace('\n','')
    new_words=new_words.replace('o/','')
    new_words=new_words.replace('I/','')
    new_words=new_words.replace('b/','')
    new_words=new_words.replace('u/','')
    new_words=new_words.replace('l/','')
    new_words=new_words.replace('n/','')
    new_words=new_words.replace('N/','')
    new_words=new_words.replace('s/','')
    new_words=new_words.replace('(','')
    new_words=new_words.replace(')','')
    new_words=new_words.replace('+',' ')
    words[0]=new_words
    # print(new_words) 
with open(txt_label_zip_toteval_root+'_processed','wb') as f:
  pickle.dump(allfile_datalist_processed,f)

In [ ]:
# if os.path.isfile(txt_label_zip_toteval_root):
with open(txt_label_zip_toteval_root+'_processed','rb') as f:
  allfile_datalist_processed=pickle.load(f)
  print(allfile_datalist_processed)

In [93]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(allfile_datalist_processed, test_size=0.25, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

14442
4815


In [94]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [95]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [96]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [97]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [98]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5)#.to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
##train
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train().to(device)
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-99-836cceb472ff>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/226 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.010582685470581 train acc 0.125
epoch 1 batch id 201 loss 0.971968948841095 train acc 0.4373445273631841
epoch 1 train acc 0.45206160450906024


<ipython-input-99-836cceb472ff>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/76 [00:00<?, ?it/s]

...

epoch 3 batch id 1 loss 0.9900408983230591 train acc 0.59375
epoch 3 batch id 201 loss 0.7515367269515991 train acc 0.6428016169154229
epoch 3 train acc 0.6469263590391909


  0%|          | 0/76 [00:00<?, ?it/s]

epoch 3 test acc 0.6065789473684211


  0%|          | 0/226 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.8779959678649902 train acc 0.671875
epoch 4 batch id 201 loss 0.5962421894073486 train acc 0.6768501243781094


In [67]:
t = torch.randn(2,2)
t.is_cuda  # returns False

# GPU에 있는 텐서
t = torch.randn(2,2).cuda()
t.is_cuda  # returns True

True